In [9]:
# !pip install -U tensorflow

In [10]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [11]:
corpus='you say goodbye and i say hello .'

### 전처리

In [12]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import numpy as np

def conver_one_hot(corpus, window_size=1):

    le = LabelEncoder()  # 숫자로 만듬
    enc = OneHotEncoder()  # one-hot 만듬
    
    corpus_list=corpus.split()
    
    le_list=le.fit_transform(corpus_list).reshape(len(corpus_list),-1)
    enc.fit(le_list)
    
    corpus_oh=enc.transform(le_list).toarray().tolist()
    
    contexts = []
    target = []
    for i, co in enumerate(corpus_oh):
        if (i < window_size) or (i >= (len(corpus_oh)-window_size)):
            continue

        contexts.append([corpus_oh[i-window_size],corpus_oh[i+window_size]])
        target.append(co)
        
    contexts = np.array(contexts)
    target = np.array(target)
    return contexts, target

In [13]:
contexts, target = conver_one_hot(corpus, 1)

In [14]:
contexts[0], target[0]

(array([[0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0.]]), array([0., 0., 0., 0., 0., 1., 0.]))

## 모델 만들기

In [15]:
# pre, post로 나누기위해 shape 변환
contexts=tf.transpose(a=contexts, perm=[1,0,2])

In [16]:
target.shape, contexts.shape

((6, 7), TensorShape([2, 6, 7]))

In [17]:
input_shape = (7,7)

In [18]:
from tensorflow.keras.layers import Input, Dense, Embedding, Dot, Reshape
from tensorflow.keras.models import Model

def Word2Vec_classification(input_shape):
    inputs1 = Input(shape=input_shape[0])
    inputs2 = Input(shape=input_shape[1])
    x1 = Dense(3, name='pre')(inputs1)  # matmul 부분
    x2 = Dense(3, name='post')(inputs2)

    dot = tf.add(x1,x2)* 0.5
        
    out = Dense(7, activation='softmax', name='output')(dot)
    
    cbow = Model(inputs=[inputs1, inputs2], outputs=out)
    
    return cbow

CBOW = Word2Vec_classification(input_shape)
CBOW.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 7)]          0                                            
__________________________________________________________________________________________________
pre (Dense)                     (None, 3)            24          input_1[0][0]                    
__________________________________________________________________________________________________
post (Dense)                    (None, 3)            24          input_2[0][0]                    
______________________________________________________________________________________________

In [19]:
CBOW.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
CBOW.fit(
    [contexts[0],contexts[1]],
    target,
    batch_size=3,
    epochs=1000
)

Train on 6 samples
Epoch 1/1000
6/6 [==============================] - 1s 107ms/sample - loss: 1.9362 - accuracy: 0.1667
Epoch 2/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9327 - accuracy: 0.1667
Epoch 3/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9294 - accuracy: 0.1667
Epoch 4/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9263 - accuracy: 0.1667
Epoch 5/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9233 - accuracy: 0.1667
Epoch 6/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9205 - accuracy: 0.1667
Epoch 7/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9172 - accuracy: 0.1667
Epoch 8/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9143 - accuracy: 0.1667
Epoch 9/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.9112 - accuracy: 0.1667
Epoch 10/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.90

6/6 [==============================] - 0s 1ms/sample - loss: 1.7126 - accuracy: 0.5000
Epoch 82/1000
6/6 [==============================] - 0s 940us/sample - loss: 1.7100 - accuracy: 0.5000
Epoch 83/1000
6/6 [==============================] - 0s 972us/sample - loss: 1.7072 - accuracy: 0.5000
Epoch 84/1000
6/6 [==============================] - 0s 977us/sample - loss: 1.7042 - accuracy: 0.5000
Epoch 85/1000
6/6 [==============================] - 0s 932us/sample - loss: 1.7016 - accuracy: 0.5000
Epoch 86/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.6987 - accuracy: 0.5000
Epoch 87/1000
6/6 [==============================] - 0s 999us/sample - loss: 1.6960 - accuracy: 0.5000
Epoch 88/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.6933 - accuracy: 0.5000
Epoch 89/1000
6/6 [==============================] - 0s 979us/sample - loss: 1.6902 - accuracy: 0.5000
Epoch 90/1000
6/6 [==============================] - 0s 947us/sample - loss: 1.6872 - accurac

6/6 [==============================] - 0s 1ms/sample - loss: 1.4753 - accuracy: 0.5000
Epoch 161/1000
6/6 [==============================] - 0s 930us/sample - loss: 1.4722 - accuracy: 0.5000
Epoch 162/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4691 - accuracy: 0.5000
Epoch 163/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4658 - accuracy: 0.5000
Epoch 164/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4626 - accuracy: 0.5000
Epoch 165/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4599 - accuracy: 0.5000
Epoch 166/1000
6/6 [==============================] - 0s 988us/sample - loss: 1.4563 - accuracy: 0.5000
Epoch 167/1000
6/6 [==============================] - 0s 992us/sample - loss: 1.4532 - accuracy: 0.5000
Epoch 168/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.4499 - accuracy: 0.5000
Epoch 169/1000
6/6 [==============================] - 0s 941us/sample - loss: 1.4467 - accu

6/6 [==============================] - 0s 1ms/sample - loss: 1.2199 - accuracy: 0.6667
Epoch 240/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.2166 - accuracy: 0.6667
Epoch 241/1000
6/6 [==============================] - 0s 959us/sample - loss: 1.2134 - accuracy: 0.6667
Epoch 242/1000
6/6 [==============================] - 0s 981us/sample - loss: 1.2102 - accuracy: 0.6667
Epoch 243/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.2070 - accuracy: 0.6667
Epoch 244/1000
6/6 [==============================] - 0s 878us/sample - loss: 1.2038 - accuracy: 0.6667
Epoch 245/1000
6/6 [==============================] - 0s 996us/sample - loss: 1.2006 - accuracy: 0.6667
Epoch 246/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.1973 - accuracy: 0.6667
Epoch 247/1000
6/6 [==============================] - 0s 993us/sample - loss: 1.1940 - accuracy: 0.6667
Epoch 248/1000
6/6 [==============================] - 0s 1ms/sample - loss: 1.1908 - ac

Epoch 319/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9676 - accuracy: 0.8333
Epoch 320/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9645 - accuracy: 0.8333
Epoch 321/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9614 - accuracy: 0.8333
Epoch 322/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9584 - accuracy: 0.8333
Epoch 323/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9555 - accuracy: 0.8333
Epoch 324/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9523 - accuracy: 0.8333
Epoch 325/1000
6/6 [==============================] - 0s 973us/sample - loss: 0.9493 - accuracy: 0.8333
Epoch 326/1000
6/6 [==============================] - 0s 987us/sample - loss: 0.9463 - accuracy: 0.8333
Epoch 327/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.9433 - accuracy: 0.8333
Epoch 328/1000
6/6 [==============================] - 0s 955us/sample - loss: 

...

6/6 [==============================] - 0s 1ms/sample - loss: 0.2896 - accuracy: 1.0000
Epoch 639/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2884 - accuracy: 1.0000
Epoch 640/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2874 - accuracy: 1.0000
Epoch 641/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2862 - accuracy: 1.0000
Epoch 642/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2851 - accuracy: 1.0000
Epoch 643/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2841 - accuracy: 1.0000
Epoch 644/1000
6/6 [==============================] - 0s 998us/sample - loss: 0.2829 - accuracy: 1.0000
Epoch 645/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2818 - accuracy: 1.0000
Epoch 646/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2809 - accuracy: 1.0000
Epoch 647/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2796 - accuracy: 

6/6 [==============================] - 0s 1ms/sample - loss: 0.2139 - accuracy: 1.0000
Epoch 719/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2132 - accuracy: 1.0000
Epoch 720/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2123 - accuracy: 1.0000
Epoch 721/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2117 - accuracy: 1.0000
Epoch 722/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2109 - accuracy: 1.0000
Epoch 723/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2100 - accuracy: 1.0000
Epoch 724/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2092 - accuracy: 1.0000
Epoch 725/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2086 - accuracy: 1.0000
Epoch 726/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2077 - accuracy: 1.0000
Epoch 727/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.2070 - accuracy: 1.

6/6 [==============================] - 0s 999us/sample - loss: 0.1611 - accuracy: 1.0000
Epoch 799/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1605 - accuracy: 1.0000
Epoch 800/1000
6/6 [==============================] - 0s 1000us/sample - loss: 0.1601 - accuracy: 1.0000
Epoch 801/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1594 - accuracy: 1.0000
Epoch 802/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1590 - accuracy: 1.0000
Epoch 803/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1584 - accuracy: 1.0000
Epoch 804/1000
6/6 [==============================] - 0s 976us/sample - loss: 0.1578 - accuracy: 1.0000
Epoch 805/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1574 - accuracy: 1.0000
Epoch 806/1000
6/6 [==============================] - 0s 994us/sample - loss: 0.1568 - accuracy: 1.0000
Epoch 807/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1563 - acc

6/6 [==============================] - 0s 961us/sample - loss: 0.1238 - accuracy: 1.0000
Epoch 879/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1234 - accuracy: 1.0000
Epoch 880/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1230 - accuracy: 1.0000
Epoch 881/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1226 - accuracy: 1.0000
Epoch 882/1000
6/6 [==============================] - 0s 982us/sample - loss: 0.1222 - accuracy: 1.0000
Epoch 883/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1219 - accuracy: 1.0000
Epoch 884/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1215 - accuracy: 1.0000
Epoch 885/1000
6/6 [==============================] - 0s 968us/sample - loss: 0.1211 - accuracy: 1.0000
Epoch 886/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1207 - accuracy: 1.0000
Epoch 887/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.1203 - accura

Epoch 958/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0969 - accuracy: 1.0000
Epoch 959/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0966 - accuracy: 1.0000
Epoch 960/1000
6/6 [==============================] - 0s 986us/sample - loss: 0.0963 - accuracy: 1.0000
Epoch 961/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0961 - accuracy: 1.0000
Epoch 962/1000
6/6 [==============================] - 0s 971us/sample - loss: 0.0958 - accuracy: 1.0000
Epoch 963/1000
6/6 [==============================] - 0s 945us/sample - loss: 0.0955 - accuracy: 1.0000
Epoch 964/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0952 - accuracy: 1.0000
Epoch 965/1000
6/6 [==============================] - 0s 988us/sample - loss: 0.0949 - accuracy: 1.0000
Epoch 966/1000
6/6 [==============================] - 0s 1ms/sample - loss: 0.0947 - accuracy: 1.0000
Epoch 967/1000
6/6 [==============================] - 0s 943us/sample - lo

### 분산표현

In [21]:
np.save('CBOW_emb.npy', CBOW.get_weights()[0])

In [22]:
W_in = np.load(open('CBOW_emb.npy', 'rb'))

In [23]:
W_in

array([[-0.40738615,  0.04770571, -0.6967565 ],
       [ 0.5827626 , -0.7436525 , -1.6604034 ],
       [ 1.0446596 ,  1.2142888 , -0.6108017 ],
       [-0.43419313, -0.68445045,  0.7027286 ],
       [-0.89910066,  0.92060506, -0.6463487 ],
       [-1.784869  , -1.607107  ,  0.604898  ],
       [-1.6441095 ,  1.2980825 , -0.30355215]], dtype=float32)

### 모델 저장

In [24]:
model_json = CBOW.to_json()
with open("CBOW_emb_scratch.json", "w") as json_file : 
    json_file.write(model_json)
    
CBOW.save_weights("CBOW_emb_scratch.h5")
print("Saved model to disk")

Saved model to disk


In [25]:
from tensorflow.keras.models import model_from_json 
with open("CBOW_emb_scratch.json", "r") as json_file : 
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("CBOW_emb_scratch.h5")
print("Loaded model from disk")

Loaded model from disk


In [26]:
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### train, test data 나누지 않음, data가 너무 적어서 모델을 만드는데 의의를 둠

In [27]:
target_pre=loaded_model.predict([contexts[0],contexts[1]])

In [28]:
def accuracy_euni(y_target, y_pred):
    y_true, pred = y_target.argmax(axis=1), y_pred.argmax(axis=1)
    
    return np.mean(pred == y_true)

In [29]:
accuracy_euni(target,target_pre)

1.0

In [30]:
target_pre

array([[3.21447238e-04, 7.21461419e-03, 6.43037958e-03, 2.00634375e-02,
        2.19531415e-04, 9.65665221e-01, 8.54210302e-05],
       [4.63987142e-03, 3.91357957e-04, 9.03529167e-01, 6.10617176e-02,
        1.15272379e-03, 2.36834884e-02, 5.54174511e-03],
       [3.82420328e-03, 9.05040443e-01, 3.35400138e-04, 2.28427979e-03,
        2.90636588e-02, 5.78256659e-02, 1.62630109e-03],
       [1.77355867e-03, 3.99958342e-02, 8.11781356e-05, 2.76955552e-02,
        9.26300526e-01, 8.78918916e-04, 3.27439746e-03],
       [8.81961780e-04, 1.87485572e-02, 1.12304045e-02, 2.21864451e-02,
        5.88295050e-04, 9.46103632e-01, 2.60784494e-04],
       [2.72652833e-03, 1.47082715e-03, 4.91653010e-02, 8.64550471e-01,
        4.34848182e-02, 3.44341770e-02, 4.16781427e-03]], dtype=float32)

In [31]:
target_pre.argmax(axis=1)

array([5, 2, 1, 4, 5, 3])

In [18]:
target

array([[0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0.]])